In [1]:
from sigvisa import Sigvisa
from sigvisa.infer.propose_lstsqr import ev_lstsqr_dist
from sigvisa.infer.event_splitmerge import sample_events_to_merge, event_merge_dist, sample_events_to_merge, merge_helper_newstyle, split_helper_newstyle
from sigvisa.infer.propose_hough import hough_location_proposal
from sigvisa.infer.event_birthdeath import prior_location_proposal, prettyprint_debug

import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

import cPickle as pickle

from sigvisa.infer.event_birthdeath import ev_bare_birth_move, ev_bare_death_move, ev_template_birth_helper, ev_template_death_helper
from sigvisa.utils.counter import Counter
from sigvisa.utils.geog import dist_km
from sigvisa.source.event import Event

# Merge/split sanity check

In [2]:
with open("/home/dmoore/python/sigvisa/notebooks/thesis/merge_test2.sg", 'rb') as f:
    sg = pickle.load(f)

In [ ]:
lp_old = sg.current_log_p()

loading model 110093
loading model 110095
loading model 110109
loading model 110098
loading model 110098
loading model 110099
loading model 110114
loading model 110116
loading model 110093
loading model 110098
loading model 110096
loading model 110113
loading model 110113
loading model 110115
loading model 110108
loading model 110115
loading model 110098
loading model 110094
loading model 110110
loading model 110114
loading model 110094
loading model 110095
loading model 110098
loading model 110110
loading model 110096
loading model 110113
loading model 110111
loading model 110100
loading model 110113
loading model 110109
loading model 110105
loading model 110104
loading model 110104
loading model 110101
loading model 110105
loading model 110115
loading model 110093
loading model 110108
loading model 110109
loading model 110110
loading model 110105
loading model 110093
loading model 110106
loading model 110101
loading model 110103
loading model 110116
loading model 110114
loading model

In [ ]:
from sigvisa.infer.event_splitmerge import ev_merge_move_hough, ev_merge_move_corr, ev_split_move_hough, ev_split_move_corr

#a = ev_merge_move_hough(sg, log_to_run_dir="/home/dmoore/python/sigvisa")
#b = ev_merge_move_corr(sg, log_to_run_dir="/home/dmoore/python/sigvisa")
c = ev_split_move_hough(sg, log_to_run_dir="/home/dmoore/python/sigvisa")
d = ev_split_move_corr(sg, log_to_run_dir="/home/dmoore/python/sigvisa")


In [5]:
def ev_split_move_hough(sg, **kwargs):

    flp, rlp, birth_args, death_args = sample_repropose_args_hough(sg)

    return split_helper_newstyle2(sg,
                                 forward_location_proposal=flp,
                                 reverse_location_proposal=rlp,
                                 birth_args=birth_args,
                                 death_args=death_args,
                                 **kwargs)

def ev_split_move_corr(sg, **kwargs):

    flp, rlp, birth_args, death_args = sample_repropose_args_corr(sg)

    return split_helper_newstyle2(sg,
                                 forward_location_proposal=flp,
                                 reverse_location_proposal=rlp,
                                 birth_args=birth_args,
                                 death_args=death_args,
                                 **kwargs)


from sigvisa.infer.mcmc_basic import mh_accept_util
from sigvisa.infer.event_swap import rebirth_events_helper, sample_repropose_args_hough, sample_repropose_args_corr
from sigvisa.infer.event_splitmerge import sample_event_to_split, sample_events_to_merge

def split_helper_newstyle2(sg,
                          log_to_run_dir=None,
                          return_probs=False,
                          fix_split_eid=None,
                          **kwargs):

    eid, lp_split = sample_event_to_split(sg, fix_result=fix_split_eid)
    old_ev = sg.get_event(eid)

    split_eids = (sg.next_eid, sg.next_eid+1)
    lp_new, lp_old, log_qforward, log_qbackward, revert_move = rebirth_events_helper(sg, (eid,),
                                                                                     birth_eids=split_eids,
                                                                                     **kwargs)
    log_qforward += lp_split
    new_evs = [sg.get_event(new_eid) for new_eid in split_eids]

    _, _, lp_merge = sample_events_to_merge(sg, fix_result=split_eids)
    log_qbackward += lp_merge

    if log_to_run_dir is not None:
        log_file = os.path.join(log_to_run_dir, "split_proposals.txt")
        with open(log_file, 'a') as f:
            f.write("splitting %d to %d %d, lp %.2f reverse %.2f\n" % (eid, split_eids[0], split_eids[1],
                                                                       lp_split, lp_merge))
            f.write("kwargs %s\n" % kwargs)

            f.write("old eid %d\n" % (eid))
            f.write("  %s\n" % str(old_ev))

            f.write("new eids %d %d\n" % split_eids)
            for new_ev in new_evs:
                f.write("  %s\n" % str(new_ev))

            f.write("lp_new %.1f lp_old %.1f log_qbackward %.1f log_qforward %.1f ratio %.1f\n\n" % (lp_new, lp_old, log_qbackward, log_qforward, lp_new+log_qbackward - (lp_old+log_qforward)))

    if return_probs:
        return lp_old, lp_new, log_qforward, log_qbackward, revert_move
    else:
        return mh_accept_util(lp_old, lp_new, log_qforward, log_qbackward, accept_move=None, revert_move=revert_move)
    
    

In [6]:


c = ev_split_move_hough(sg, log_to_run_dir="/home/dmoore/python/sigvisa")
d = ev_split_move_corr(sg, log_to_run_dir="/home/dmoore/python/sigvisa")


ELK
PD31
NV01
NEW
global hough: init 0.066061 stahough 6.455276 adj 0.012861 total 6.534219
-123.563333333
-123.2925
ELK
PD31
NV01
NEW
global hough: init 0.000011 stahough 0.032254 adj 0.000018 total 0.032292
-123.563333333
-123.427916667
ELK
PD31
NV01
NEW
global hough: init 0.000011 stahough 0.043784 adj 0.000020 total 0.043823
repropose death 19 lqf -1.216041 lqb -13.794038
prior sampled ev evid None, loc 122.02 W 47.31 N, depth 8.9km, time 1239043208.9, mb 3.1, natural source


/home/dmoore/.virtualenvs/sigvisa/local/lib/python2.7/site-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


ValueError: adding node '154;mb' to the graph, but a node with this label already exists!

In [4]:
from sigvisa.infer.event_birthdeath import prior_location_proposal
from sigvisa.source.event import Event

def fixed_lp(sg, fix_result=None, proposal_dist_seed=None):
    fixed_ev = Event(lon=-116.40, lat=33.25, depth=0.4, time=1239042392.4, mb=4.3)
    if fix_result is not None:
        return 0.0
    else:
        return fixed_ev, 0.0, None
    


In [16]:

dumb_birth = False
birth_args = {"proposal_type": "dumb" if dumb_birth else "mh",
              "use_correlation": True,
              "repropose_uatemplates": True}

dumb_death = True
death_args = {"birth_type": "dumb" if dumb_death else "mh",
              "use_correlation": not dumb_death,
              "repropose_uatemplates": not dumb_death}


r = merge_move_newstyle(sg, return_probs=True, 
                        fix_merge_eids=(43,19), 
                        forward_location_proposal=fixed_lp,
                        reverse_location_proposal=prior_location_proposal,
                        birth_args=birth_args,
                        death_args=death_args)
lp_old, lp_new, log_qforward, log_qbackward, revert_move = r


prior sampled ev evid None, loc 123.52 W 40.80 N, depth 4.0km, time 1239042405.1, mb 3.6, natural source
repropose death 19 lqf -0.409407 lqb -47.978365
prior sampled ev evid None, loc 121.50 W 35.65 N, depth 3.4km, time 1239042418.6, mb 3.3, natural source
repropose death 43 lqf -1.037745 lqb -73.747009
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
proposal using correlation True
repropose birth 19 lqf 49.705055 lqb -76.198992


In [17]:
print lp_new
lp_old, lp_new, log_qforward, log_qbackward, revert_move = r
log_qbackward = log_qbackward[1]

print lp_old, lp_new, log_qforward, log_qbackward
print lp_new - lp_old
print (lp_new + log_qbackward) - (lp_old + log_qforward)


-599092.956358
-599101.119692 -599092.956358 46.3934465914 -199.870276565
8.1633343756
-238.100388781


In [18]:
import cPickle as pickle
with open("/home/dmoore/python/sigvisa/logs/mcmc/01593/merged.sg", 'wb') as f:
    pickle.dump(sg, f, 2)

In [19]:
revert_move()

In [20]:
from sigvisa.infer.event_splitmerge import event_merge_dist

evm = event_merge_dist(sg)

In [21]:
print evm

{(19, 136): 0.022721909027234373, (70, 73): 0.022721909027234373, (19, 70): 0.022721909027234373, (9, 34): 0.043012121394611545, (19, 73): 0.022721909027234373, (9, 136): 0.022721909027234373, (9, 22): 0.022721909027234373, (19, 22): 0.022721909027234373, (34, 73): 0.022721909027234373, (9, 70): 0.022721909027234373, (34, 70): 0.022721909027234373, (70, 136): 0.23395974056004132, (34, 136): 0.022721909027234373, (43, 73): 0.022721909027234373, (34, 43): 0.022721909027234373, (9, 73): 0.022721909027234373, (43, 136): 0.022721909027234373, (19, 43): 0.15498041236448803, (22, 34): 0.022721909027234373, (73, 136): 0.022721909027234373, (22, 73): 0.022721909027234373, (9, 43): 0.022721909027234373, (22, 70): 0.022721909027234373, (43, 70): 0.022721909027234373, (19, 34): 0.022721909027234373, (22, 136): 0.022721909027234373, (22, 43): 0.022721909027234373, (9, 19): 0.022721909027234373}


In [6]:
lp_new = sg.current_log_p()

# Merge/split acceptance debugging

In [4]:
with open("/home/dmoore/python/sigvisa/notebooks/thesis/merge_test.sg", 'rb') as f:
    sg = pickle.load(f)

In [7]:
lp_old = sg.current_log_p()
print "lp_old", lp_old
true_ev = Event(lon=-118.36, lat=39.14, depth=1.7, time=1239041021.9, mb=3.2)

merge_debug = {}
lqf, replicate_merge, merge_record = merge_events_helper(sg, hough_location_proposal, 
                                                         force_eids=(6, 49),
                                                         force_new_ev=true_ev,
                                                         debug_info=merge_debug,
                                                         birth_proposal_type="mh")

lp_new = sg.current_log_p()
print "lp_new", lp_new

debug_info = {}
lqb, replicate_split, split_record = split_events_helper(sg, prior_location_proposal, 
                                                         birth_proposal_type="dumb", 
                                                         debug_info = debug_info,
                                                         fix_result=merge_record,
                                                         
                                                         flip_ordering=True)
lp_old2 = sg.current_log_p()

print lp_old, lp_old2

lp_old -271699.051097
merging eids 6 49
baseline lp -271605.580841
YKR8
ANMO
KDAK
PFO
TX01
YBH
NV01
IL31
PD31
ELK
ULM
NEW
global hough: init 0.014510 stahough 3.777926 adj 0.112285 total 3.904770
-118.808510638
-118.531914894
YKR8
ANMO
KDAK
PFO
TX01
YBH
NV01
IL31
PD31
ELK
ULM
NEW
global hough: init 0.000009 stahough 0.120007 adj 0.000051 total 0.120082
-118.531914894
-118.393617021
YKR8
ANMO
KDAK
PFO
TX01
YBH
NV01
IL31
PD31
ELK
ULM
NEW
global hough: init 0.000009 stahough 0.044173 adj 0.000046 total 0.044241
merge lqf choice -0.0726305136695 death1 -0.525032172135 death2 -0.133818707147 loc 2.81725656643 tmpls -118.313816265
merge lqf totals -116.228041091 -116.228041091
lp_new -271510.192621
baseline lp -271605.580841
split lps -0.69314718056 -11.9075362279 -17.9991697227 -10.5583925282 -240.577042706 -175.716946053 total -458.145381599 -457.452234419
-271699.051097 -271699.051097


In [8]:
print lp_new, lp_old

print lp_new + lqb
print lp_old + lqf

print "merge birth"
print prettyprint_debug(merge_debug)

dbg1 = debug_info["ev1"]
dbg2 = debug_info["ev2"]
print 
print "split birth 1"
print prettyprint_debug(dbg1)
print
print "split birth 2"
print prettyprint_debug(dbg2)

-271510.192621 -271699.051097
-271968.338003
-271815.279138
merge birth
proposed ev evid None, loc 118.36 W 39.14 N, depth 1.7km, time 1239041021.9, mb 3.2, natural source
proposal logq 2.82, lps loc -2.31, source -0.00, mb -0.78, time 0.00, nevents -11.38, total -14.46, delta -17.28
sta ELK wn_old -18322.88 wn_new -18277.41 wn_delta 45.47 assoc -0.01 overall 62.21
 phase Lg: associating tmid 10730, score 7.76
   amp_transfer 7.71 ualp -2.22 evlp -1.17 delta 1.05
   coda_decay -2.16 ualp -1.35 evlp -1.02 delta 0.32
   mult_wiggle_std 0.97 ualp 0.63 evlp 1.25 delta 0.62
   peak_decay -4.55 ualp -3.02 evlp -2.28 delta 0.74
   peak_offset 1.51 ualp -1.62 evlp -1.65 delta -0.03
   tt_residual -8.25 ualp -9.21 evlp -4.15 delta 5.06
 phase P: birthing new template, score -9.22
   amp_transfer 6.31 logq 0.49 logp -1.14 delta -1.63
   coda_decay -4.12 logq 1.15 logp -0.87 delta -2.02
   mult_wiggle_std 0.36 logq 1.03 logp 0.46 delta -0.58
   peak_decay -3.68 logq -1.23 logp -1.29 delta -0.06
 


location_proposal = hough_location_proposal
proposal_includes_mb=True
use_correlation=False
birth_proposal_type="smart"
fix_result=None

log_qforward = 0.0
merge_record = {}

fr_eids = fix_result["eids"] if fix_result is not None else None
if fr_eids is not None and (fr_eids[0] < 0 or fr_eids[1] < 0):
    # if the location proposal in the split move failed, then one
    # or both events were not created and so don't now need to be
    # killed.
    log_qforward = -np.inf

    replicate_untmpls1 = noop
    replicate_death1 = noop

    replicate_untmpls2 = noop
    replicate_death2 = noop

else:
    eid1, eid2, merge_lp = sample_events_to_merge(sg, fix_result=fr_eids)
    #if eid1 is None:
    #    # not enough events for a merge move
    #    return None, None, None

    merge_record["merged_ev1"] = (sg.get_event(eid1), eid1)
    merge_record["merged_ev2"]= (sg.get_event(eid2), eid2)
    log_qforward += merge_lp

    # remove both merged events
    fr_eid1 = fix_result["split_tmpls1"] if fix_result is not None else None
    lqf1, replicate_untmpls1, death_records1 = ev_template_death_helper(sg, eid1, fix_result=fr_eid1)
    merge_record["merged_tmpls1"] = death_records1
    log_qforward += lqf1
    replicate_death1, ev1 = ev_bare_death_move(sg, eid1)

    fr_eid2 = fix_result["split_tmpls2"] if fix_result is not None else None
    lqf2, replicate_untmpls2, death_records2 = ev_template_death_helper(sg, eid2, fix_result=fr_eid2)
    merge_record["merged_tmpls2"] = death_records2
    log_qforward += lqf2
    replicate_death2, ev2 = ev_bare_death_move(sg, eid2)
    sg._topo_sort()
    
lp_baseline = sg.current_log_p()
lqf_mergedeath = log_qforward
    
debug_info = {}
fr_ev = fix_result["split_ev"] if fix_result is not None else None
lq_loc, replicate_birth, eid, extra = ev_bare_birth_move(sg, location_proposal, debug_info=debug_info, fix_result=fr_ev)
if lq_loc is None:
    # hack: if location proposal fails, just don't birth an
    # event so that the "merge" becomes a double death move (which
    # will presumably fail). This means we have to be careful to
    # ensure that the split move effectively reverses the effects
    # of the merge-so-far.
    merge_record["merged_eid"] = -1
    replicate_birth = noop
    replicate_tmpls = noop
else:
    merge_record["merged_eid"] = eid
    log_qforward += lq_loc


    fr_birth = fix_result["split_tmpls"] if fix_result is not None else None
    lqf, replicate_tmpls, birth_records = \
                ev_template_birth_helper(sg, eid,
                                         associate_using_mb=proposal_includes_mb,
                                         use_correlation=use_correlation,
                                         proposal_type=birth_proposal_type,
                                         debug_info=debug_info,
                                         fix_result=fr_birth)
    log_qforward += lqf
    sg._topo_sort()
    merge_record["merged_tmpls"] = birth_records

def replicate_merge():
    replicate_untmpls1()
    replicate_untmpls2()
    replicate_death1()
    replicate_death2()
    sg._topo_sort()
    replicate_birth()
    replicate_tmpls()
    sg._topo_sort()

merge_qforward = log_qforward
lqf_mergebirth = lqf

In [5]:
lp_new = sg.current_log_p()

In [6]:
print lp_old,  lp_new



-271699.051097 -278437.128168


In [7]:
sg.current_log_p_breakdown()

n_uatemplate: -188.5
n_event: -22.8
ev priors: ev -12.0
ev observations: ev 0.0
phase existence: ev -11.6
tt_residual: ev -344.7
ev global cost (n + priors + phases + tt): -391.1
coda_decay: ev -150.6 ua -35.1 total -185.8
peak_decay: ev -118.7 ua -28.6 total -147.3
peak_offset: ev -101.2 ua -36.6 total -137.8
coda_height: ev -140.6 ua -44.2 total -184.8
mult_std_wiggle: ev 26.8 ua -3.2 total 23.6
coef jointgp: 0.0
total param: ev -484.4 ua -147.7 total -632.1
priors+params: ev -875.4 ua -336.2 total -1211.7
station noise (observed signals): -277126.3
noise model prior lp: -99.1
overall: -278437.1
official: -278437.1



'n_uatemplate: -188.5\nn_event: -22.8\nev priors: ev -12.0\nev observations: ev 0.0\nphase existence: ev -11.6\ntt_residual: ev -344.7\nev global cost (n + priors + phases + tt): -391.1\ncoda_decay: ev -150.6 ua -35.1 total -185.8\npeak_decay: ev -118.7 ua -28.6 total -147.3\npeak_offset: ev -101.2 ua -36.6 total -137.8\ncoda_height: ev -140.6 ua -44.2 total -184.8\nmult_std_wiggle: ev 26.8 ua -3.2 total 23.6\ncoef jointgp: 0.0\ntotal param: ev -484.4 ua -147.7 total -632.1\npriors+params: ev -875.4 ua -336.2 total -1211.7\nstation noise (observed signals): -277126.3\nnoise model prior lp: -99.1\noverall: -278437.1\nofficial: -278437.1\n'

In [8]:
lp_new = sg.current_log_p()

debug_info = {}
lqb, replicate_split, split_record = split_events_helper(sg, prior_location_proposal, 
                                                         birth_proposal_type="dumb", 
                                                         debug_info = debug_info,
                                                         fix_result=merge_record,
                                                         flip_ordering=False)

baseline lp -271605.580841
split lps -0.69314718056 -7.64185652484 -17.9991697227 -10.5583925282 -185.658882435 -224.175767982 total -447.420363554 -446.727216373


In [9]:
print lp_new-lp_old
print log_qforward
print lqb

-6738.07707168
-307.86160771
-447.420363554


In [30]:
lp = 0.0
score = 0.0

lp = debug_info["ev2"]["ev"][1]

for sta, stadict in debug_info["ev2"].items():
    if sta.startswith("ev"): continue
    
    wndiff = stadict["wn_new"] - stadict["wn_old"]
    score += wndiff
    #lp += wndiff
    
    assocprob = stadict["assoc"][1]
    lp += assocprob
    score += assocprob
    
    print sta, "wn", wndiff
    print sta, "assoc", assocprob
    for phase, pdict in stadict.items():
        if phase.startswith("wn") or phase.startswith("assoc"): continue 
       
        if isinstance(pdict, dict):
            plp =0.0
            for param, (v, lqq, lq) in pdict.items():
                score += lq - lqq
                if np.abs(lq-lqq) > 1e-6:
                    print sta, phase, param, v, lqq, lq
                plp += lq
            print sta, phase, "param q", plp
            
            lp += plp
        else:
            print sta, phase, pdict

    print "lp", lp
    
print "score", score

ELK wn 24.9855866455
ELK assoc -0.00744403193634
ELK Lg mult_wiggle_std 0.972943642855 0.631394723412 1.24731280478
ELK Lg amp_transfer 8.09571782359 -2.22358661899 -1.15999803279
ELK Lg coda_decay -2.16316812063 -1.34961602242 -1.02903481063
ELK Lg peak_offset 1.50861593112 -1.61786543571 -1.68591988002
ELK Lg tt_residual -8.12769826581 -9.21034037198 -4.12117395445
ELK Lg peak_decay -4.55153745562 -3.02334149912 -2.40043979822
ELK Lg param q -9.14925367134
ELK P param q -8.9824471546
ELK S mult_wiggle_std 0.499944934392 -0.297661016377 0.626982317743
ELK S amp_transfer 7.80618788617 -2.16629124635 -1.73830985533
ELK S coda_decay -2.94186772173 -1.36779188209 -1.30159382327
ELK S peak_offset -0.999585994119 -1.71215109157 0.860960050928
ELK S tt_residual 0.308992067686 -9.21034037198 -2.8507137395
ELK S peak_decay -2.27155130517 -0.94503293629 -1.02311932819
ELK S param q -5.42579437761
ELK Pg mult_wiggle_std 0.746022275674 0.71228325115 0.605152147351
ELK Pg amp_transfer 8.3295020765

In [19]:
lp_old2 = sg.current_log_p()

In [20]:
print lp_old, lp_old2

-271699.051097 -271699.051097


# Split/Merge Sanity check

In [32]:
with open("/home/dmoore/python/sigvisa/notebooks/thesis/merge_test.sg", 'rb') as f:
    sg = pickle.load(f)

In [33]:
debug_info = {}
lp_old = sg.current_log_p()

lqf, replicate_split, split_record = split_events_helper(sg, prior_location_proposal, 
                                                         birth_proposal_type="dumb", 
                                                         debug_info = debug_info,                                                         
                                                         flip_ordering=False)
lp_new = sg.current_log_p()

lqb, replicate_merge, merge_record = merge_events_helper(sg, prior_location_proposal, 
                                                         birth_proposal_type="dumb",
                                                         fix_result=split_record)
lp_old2 = sg.current_log_p()




baseline lp -271570.580607
split lps -1.09861228867 -0.525032172135 -12.8629426877 -9.58016607532 -296.678212876 -299.014617421 total -620.452730701 -619.759583521
baseline lp -271570.580607
